# Generators use opmath

In [1]:
import pennylane as qml

FRISUS LOG: transform e' stata chiamata
FRISUS LOG: transform e' stata chiamata
FRISUS LOG: transform e' stata chiamata
FRISUS LOG: transform e' stata chiamata
FRISUS LOG: transform e' stata chiamata
FRISUS LOG: transform e' stata chiamata
FRISUS LOG: transform e' stata chiamata
FRISUS LOG: transform e' stata chiamata
FRISUS LOG: transform e' stata chiamata
FRISUS LOG: transform e' stata chiamata
FRISUS LOG: transform e' stata chiamata
FRISUS LOG: transform e' stata chiamata
FRISUS LOG: transform e' stata chiamata
FRISUS LOG: transform e' stata chiamata
FRISUS LOG: transform e' stata chiamata
FRISUS LOG: transform e' stata chiamata
FRISUS LOG: transform e' stata chiamata
FRISUS LOG: transform e' stata chiamata
FRISUS LOG: transform e' stata chiamata
FRISUS LOG: transform e' stata chiamata
FRISUS LOG: transform e' stata chiamata
FRISUS LOG: transform e' stata chiamata
FRISUS LOG: transform e' stata chiamata
FRISUS LOG: transform e' stata chiamata
FRISUS LOG: transform e' stata chiamata


## Story's description

More context: https://xanaduhq.slack.com/archives/C06DX049G49/p1705355151489519

Some operators have a generator method and we want to make sure that they are using the new opmath (in particular, no Tensor or Hamiltonian instances)

AC:

no generator method uses hardcoded Hamiltonian or Tensor instances

### I copy and paste here the *Slack* description of the thread in which the story was born

**Christina Lee**
I think we also need to switch all the generators to `Prod` as well.

**Korbinian Kottmann**
Can you elaborate, what do you mean by switching the generators? (edited) 

**Korbinian Kottmann**
Like switch definitions like these to use s_prod, prod, and sum instead of *, @ and + for construction?
https://github.com/PennyLaneAI/pennylane/blob/master/pennylane/ops/qubit/qchem_ops.py#L316
qchem_ops.py
    def generator(self):
<https://github.com/PennyLaneAI/pennylane|PennyLaneAI/pennylane>PennyLaneAI/pennylane | Added by GitHub

**Christina Lee**
The generators might be hard coded to tensor or hamiltonian.

**Korbinian Kottmann**
Do you have an example? Would the example above constitute as such by having the generator defined as scalar * (op1 @ op2) ?
qchem_ops.py
    def generator(self):
<https://github.com/PennyLaneAI/pennylane|PennyLaneAI/pennylane>PennyLaneAI/pennylane | Added by GitHub

**Christina Lee**
`DoubleExcitation` is returning a Hamiltonian.

**Korbinian Kottmann**
Thanks!

**Korbinian Kottmann**
Created a story so we dont forget about this (n.d.r.: this one)

## Primi pensieri

Penso che vogliano che io controlli che i `generators` (qualunque cosa essi siano) degli `operators` stiano usando le funzioni del modulo `pennylane/ops/opmath`, visto che Tensor e Hamiltonian sono in fase di deprecazione.

Alcune domande:

- Come faccio a capire quali `generators` devo guardare?
- Come faccio a sapere se il `generator` che sto guardando deve essere modificato? La TZ parlava di `DoubleExcitation` (qualunque cosa sia), quindi potrei partire from there

### Deeper description:

Many operators have a generator method defined, e.g.
https://github.com/PennyLaneAI/pennylane/blob/master/pennylane/ops/qubit/qchem_ops.py#L598

The task of this story is to make sure that these returns new opmath instances `(SProd, Prod, Sum)`. In particular, currently the default for python operators `@` , `+` and `*` is to use "old opmath" and creates `Tensor` and `Hamiltonian` instances.

So the task is basically to change in all `Operator.generator` definitions the following:

`x * op` -> `qml.s_prod(x, op)` , `x+y` -> `qml.sum(x, y)` , `x @ y` -> `qml.prod(x, y)` and `qml.Hamiltonian(coeffs, ops)` -> `qml.dot(coeffs, ops)` 

then everything should be (nested) instances of `SProd` , `Prod` and `Sum` - the new opmath classes.

I'd approach the task as follows:
repo-search (e.g. in visual code with cmd-shift-f) `def generator(` in `pennylane/pennylane/` and make the above changes.

Check if tests fail/pass.

If fail if the test just needs an update (e.g. changing and `isinstance(op, Hamiltonian)` to `isinstance(op, Sum)`)

If there is no failing test to change, consider updating the test of the generator method to check `isinstance(op, Sum)` (or whichever class the result is). Ideally if there already a test for the generator method in the class, update this one. If it doesnt fit, add a new one

- Changed `/ops/qubit/parametric_ops_multi_qubit.py`, line 776 

    from:

    ```
    def generator(self):
        return -0.5 * PauliX(wires=self.wires[0]) @ PauliX(wires=self.wires[1])
    ```    
    to:

    ```
    def generator(self):
        return qml.s_prod(-0.5, qml.prod(PauliX(wires=self.wires[0]), PauliX(wires=self.wires[1])))
    ```    

- Stessa modifica alla linea 912           

- Stessa modifica alla linea 1055         

- Changed line 1238 

    from :

    ```
    def generator(self):
        return 0.25 * (
            PauliX(wires=self.wires[0]) @ PauliX(wires=self.wires[1])
            + PauliY(wires=self.wires[0]) @ PauliY(wires=self.wires[1])
        )

    ```

    to :

    ```
    def generator(self):
        return qml.s_prod(
            0.25,
            qml.sum(
                qml.prod(PauliX(wires=self.wires[0]), PauliX(wires=self.wires[1])),
                qml.prod(PauliY(wires=self.wires[0]), PauliY(wires=self.wires[1])),
            ),
        )
    ```

STOP: devo interrompere questo reportage altrimenti ci metto troppo       

In [27]:
op = qml.RX(0.2, [0])
op = qml.pow(op, 5)
op.generator()

FRISUS LOG: pow generator chiamato


5 * -0.5 * X(0)

In [24]:
op.generator()

FRISUS LOG: pow generator chiamato


5 * -0.5 * X(0)

In [4]:
op = qml.pow(op, 5)

In [5]:
op.generator()

FRISUS LOG: pow generator chiamato


5 * -0.5 * X(0)

In [6]:
op = qml.CPhaseShift00(1.23, wires=[1, 2])

In [7]:
type(op.generator())

pennylane.ops.qubit.observables.BasisStateProjector

In [8]:
op = qml.s_prod(4, qml.RX(0.2, [0]))
print(op)

4 * RX(0.2, wires=[0])


In [9]:
op = qml.simplify(qml.s_prod(5, op))
print(op)

20 * RX(0.2, wires=[0])


In [10]:
op.simplify()

20 * RX(0.2, wires=[0])

In [11]:
qml.prod(qml.X(0), qml.Y(0))

X(0) @ Y(0)

In [12]:
w0, w1, w2, w3 = [0, 1, 2, 3]
coeffs = [0.0625, 0.0625, -0.0625, 0.0625, -0.0625, 0.0625, -0.0625, -0.0625]
obs = [
    qml.X(w0) @ qml.X(w1) @ qml.X(w2) @ qml.Y(w3),
    qml.X(w0) @ qml.X(w1) @ qml.Y(w2) @ qml.X(w3),
    qml.X(w0) @ qml.Y(w1) @ qml.X(w2) @ qml.X(w3),
    qml.X(w0) @ qml.Y(w1) @ qml.Y(w2) @ qml.Y(w3),
    qml.Y(w0) @ qml.X(w1) @ qml.X(w2) @ qml.X(w3),
    qml.Y(w0) @ qml.X(w1) @ qml.Y(w2) @ qml.Y(w3),
    qml.Y(w0) @ qml.Y(w1) @ qml.X(w2) @ qml.Y(w3),
    qml.Y(w0) @ qml.Y(w1) @ qml.Y(w2) @ qml.X(w3),
]
op = qml.dot(coeffs, obs)
# op = qml.Hamiltonian(coeffs, obs)

In [13]:
coeff = op.terms()[0]

In [14]:
ops = op.terms()[1]

In [15]:
type(ops[0])

pennylane.ops.op_math.prod.Prod

In [16]:
type(obs[0])

pennylane.operation.Tensor

In [17]:
type(ops)

list

In [18]:
ops = [
    qml.X(w0) @ qml.X(w1) @ qml.X(w2) @ qml.Y(w3),
    qml.X(w0) @ qml.X(w1) @ qml.Y(w2) @ qml.X(w3),
    qml.X(w0) @ qml.Y(w1) @ qml.X(w2) @ qml.X(w3),
    qml.X(w0) @ qml.Y(w1) @ qml.Y(w2) @ qml.Y(w3),
    qml.Y(w0) @ qml.X(w1) @ qml.X(w2) @ qml.X(w3),
    qml.Y(w0) @ qml.X(w1) @ qml.Y(w2) @ qml.Y(w3),
    qml.Y(w0) @ qml.Y(w1) @ qml.X(w2) @ qml.Y(w3),
    qml.Y(w0) @ qml.Y(w1) @ qml.Y(w2) @ qml.X(w3),
]

In [19]:
ops

[X(0) @ X(1) @ X(2) @ Y(3),
 X(0) @ X(1) @ Y(2) @ X(3),
 X(0) @ Y(1) @ X(2) @ X(3),
 X(0) @ Y(1) @ Y(2) @ Y(3),
 Y(0) @ X(1) @ X(2) @ X(3),
 Y(0) @ X(1) @ Y(2) @ Y(3),
 Y(0) @ Y(1) @ X(2) @ Y(3),
 Y(0) @ Y(1) @ Y(2) @ X(3)]

In [20]:
type(ops)

list

In [21]:
qml.Hamiltonian(coeffs=coeff, observables=ops)

  (-0.0625) [X0 Y1 X2 X3]
+ (-0.0625) [Y0 X1 X2 X3]
+ (-0.0625) [Y0 Y1 X2 Y3]
+ (-0.0625) [Y0 Y1 Y2 X3]
+ (0.0625) [X0 X1 X2 Y3]
+ (0.0625) [X0 X1 Y2 X3]
+ (0.0625) [X0 Y1 Y2 Y3]
+ (0.0625) [Y0 X1 Y2 Y3]


In [22]:
type(op.generator())

FRISUS LOG: base generator chiamato!


GeneratorUndefinedError: Operation Sum does not have a generator

In [ ]:
pot = qml.pow(op, -0.5)

In [ ]:
pot.generator()

FRISUS LOG: pow generator chiamato


-0.5 * (0.0625 * (X(0) @ X(1) @ X(2) @ Y(3)) + 0.0625 * (X(0) @ X(1) @ Y(2) @ X(3)) + -0.0625 * (X(0) @ Y(1) @ X(2) @ X(3)) + 0.0625 * (X(0) @ Y(1) @ Y(2) @ Y(3)) + -0.0625 * (Y(0) @ X(1) @ X(2) @ X(3)) + 0.0625 * (Y(0) @ X(1) @ Y(2) @ Y(3)) + -0.0625 * (Y(0) @ Y(1) @ X(2) @ Y(3)) + -0.0625 * (Y(0) @ Y(1) @ Y(2) @ X(3)))

In [ ]:
op = qml.s_prod(5, qml.RX(0.3, [0]) + qml.RX(0.7, [2],))

In [ ]:
op.terms()

([5], [RX(0.3, wires=[0]) + RX(0.7, wires=[2])])

## Another error to address later on

In [ ]:
from pennylane.qchem.tapering import (
    _kernel,
    _reduced_row_echelon,
    clifford,
    optimal_sector,
    taper_hf,
    taper_operation,
    _split_pauli_sentence,
    _taper_pauli_sentence,
)

from pennylane import numpy as np

"""
 @pytest.mark.parametrize(
    ("operation", "op_wires", "op_gen"),
    [
        (
            lambda phi, wires: qml.QubitUnitary(
                qml.math.array(
                    [
                        [qml.math.cos(phi / 2), 0, 0, -1j * qml.math.sin(phi / 2)],
                        [0, qml.math.cos(phi / 2), -1j * qml.math.sin(phi / 2), 0],
                        [0, -1j * qml.math.sin(phi / 2), qml.math.cos(phi / 2), 0],
                        [-1j * qml.math.sin(phi / 2), 0, 0, qml.math.cos(phi / 2)],
                    ]
                ),
                wires=wires,
            ),
            [0, 2],
            lambda phi, wires: qml.Hamiltonian(
                [-0.5 * phi], [qml.PauliX(wires=wires[0]) @ qml.PauliX(wires=wires[1])]
            ),
        ),
    ],
)
"""
phi = 0.1

operation = qml.math.cos(phi / 2), 0, 0, -1j * qml.math.sin(phi / 2)

op_wires = ([0, 2],)


def test_taper_matrix_ops(op_wires=[0, 2]):
    """Test that taper_operation can be used with gate operation built using matrices"""

    op_wires = [0, 2]

    symbols, geometry, charge = (
        ["He", "H"],
        np.array([[0.0, 0.0, 0.0], [0.0, 0.0, 1.4588684632]]),
        1,
    )
    mol = qml.qchem.Molecule(symbols, geometry, charge)
    hamiltonian = qml.qchem.diff_hamiltonian(mol)(geometry)

    generators = qml.symmetry_generators(hamiltonian)
    paulixops = qml.paulix_ops(generators, len(hamiltonian.wires))
    paulix_sector = optimal_sector(hamiltonian, generators, mol.n_electrons)
    wire_order = hamiltonian.wires

    print(f"generators = {generators}")
    print(f"type(generators) = {generators}")

    taper_op1 = taper_operation(
        qml.IsingXX,
        generators,
        paulixops,
        paulix_sector,
        wire_order,
        op_wires=op_wires,
    )
    assert callable(taper_op1)

In [ ]:
test_taper_matrix_ops()

generators = [<Hamiltonian: terms=1, wires=[0, 2]>, <Hamiltonian: terms=1, wires=[1, 3]>]
type(generators) = [<Hamiltonian: terms=1, wires=[0, 2]>, <Hamiltonian: terms=1, wires=[1, 3]>]
FRISUS LOG: _generator_hamiltonian chiamata. gen = -0.5 * (X(0) @ X(2))
FRISUS LOG: type(gen) = <class 'pennylane.ops.op_math.sprod.SProd'>


NotImplementedError: Conversion of generator -0.5 * (X(0) @ X(2)) to Hamiltonian is currently not implemented.

In [ ]:
(2.0 * qml.sum(qml.X(0), qml.Z(0))).terms()

([2.0], [X(0) + Z(0)])

In [ ]:
gen = qml.X(0) + qml.X(2)

In [ ]:
gen.ops

In [ ]:
gen.arithmetic_depth

1

In [ ]:
type(gen)

pennylane.ops.op_math.sum.Sum

In [ ]:
gen.base

AttributeError: 'Prod' object has no attribute 'base'

In [ ]:
type(gen.base)

pennylane.ops.op_math.prod.Prod

In [ ]:
gen.base[0] @ gen.base[1]

X(0) @ X(2)

In [ ]:
type(gen[0] + gen[1])

pennylane.ops.qubit.hamiltonian.Hamiltonian

NameError: name 'm' is not defined

In [ ]:
qml.Hamiltonian([gen.scalar], [qml.X(0) @ qml.X(2)])

  (-0.5) [X0 X2]


In [ ]:
gen = gen.scalar * gen.base

In [ ]:
type(gen)

pennylane.ops.op_math.sprod.SProd

In [ ]:
gen.base.hyperparameters

{}

In [ ]:
gen = -0.5 * (qml.X(0) @ qml.X(2))

In [ ]:
type(gen)

pennylane.ops.qubit.hamiltonian.Hamiltonian

In [ ]:
op = qml.RZ(1.0, wires=[3])

## This is just a matter of style, ma ci sono priorities ben more urgent!

In [ ]:
wires = [0, 1]

w1, w2 = wires

In [ ]:
w0, w1, w2, w3 = [0, 1, 2, 3]
coeffs = [0.0625, 0.0625, -0.0625, 0.0625, -0.0625, 0.0625, -0.0625, -0.0625]
obs = [
    qml.X(w0) @ qml.X(w1) @ qml.X(w2) @ qml.Y(w3),
    qml.X(w0) @ qml.X(w1) @ qml.Y(w2) @ qml.X(w3),
    qml.X(w0) @ qml.Y(w1) @ qml.X(w2) @ qml.X(w3),
    qml.X(w0) @ qml.Y(w1) @ qml.Y(w2) @ qml.Y(w3),
    qml.Y(w0) @ qml.X(w1) @ qml.X(w2) @ qml.X(w3),
    qml.Y(w0) @ qml.X(w1) @ qml.Y(w2) @ qml.Y(w3),
    qml.Y(w0) @ qml.Y(w1) @ qml.X(w2) @ qml.Y(w3),
    qml.Y(w0) @ qml.Y(w1) @ qml.Y(w2) @ qml.X(w3),
]
Ham = qml.Hamiltonian(coeffs, obs)

In [ ]:
Ham.coeffs

[0.0625, 0.0625, -0.0625, 0.0625, -0.0625, 0.0625, -0.0625, -0.0625]

In [ ]:
Rep = qml.sum(
    qml.s_prod(0.0625, qml.X(w0) @ qml.X(w1) @ qml.X(w2) @ qml.Y(w3)),
    qml.s_prod(0.0625, qml.X(w0) @ qml.X(w1) @ qml.Y(w2) @ qml.X(w3)),
    qml.s_prod(-0.0625, qml.X(w0) @ qml.Y(w1) @ qml.X(w2) @ qml.X(w3)),
    qml.s_prod(0.0625, qml.X(w0) @ qml.Y(w1) @ qml.Y(w2) @ qml.Y(w3)),
    qml.s_prod(-0.0625, qml.Y(w0) @ qml.X(w1) @ qml.X(w2) @ qml.X(w3)),
    qml.s_prod(0.0625, qml.Y(w0) @ qml.X(w1) @ qml.Y(w2) @ qml.Y(w3)),
    qml.s_prod(-0.0625, qml.Y(w0) @ qml.Y(w1) @ qml.X(w2) @ qml.Y(w3)),
    qml.s_prod(-0.0625, qml.Y(w0) @ qml.Y(w1) @ qml.Y(w2) @ qml.X(w3)),
)

In [ ]:
Rep.terms

<bound method Sum.terms of (
    0.0625 * (X(0) @ X(1) @ X(2) @ Y(3))
  + 0.0625 * (X(0) @ X(1) @ Y(2) @ X(3))
  + -0.0625 * (X(0) @ Y(1) @ X(2) @ X(3))
  + 0.0625 * (X(0) @ Y(1) @ Y(2) @ Y(3))
  + -0.0625 * (Y(0) @ X(1) @ X(2) @ X(3))
  + 0.0625 * (Y(0) @ X(1) @ Y(2) @ Y(3))
  + -0.0625 * (Y(0) @ Y(1) @ X(2) @ Y(3))
  + -0.0625 * (Y(0) @ Y(1) @ Y(2) @ X(3))
)>

In [ ]:
coeffs = [0.0625, 0.0625, -0.0625, 0.0625, -0.0625, 0.0625, -0.0625, -0.0625]
obs = [
    qml.X(w0) @ qml.X(w1) @ qml.X(w2) @ qml.Y(w3),
    qml.X(w0) @ qml.X(w1) @ qml.Y(w2) @ qml.X(w3),
    qml.X(w0) @ qml.Y(w1) @ qml.X(w2) @ qml.X(w3),
    qml.X(w0) @ qml.Y(w1) @ qml.Y(w2) @ qml.Y(w3),
    qml.Y(w0) @ qml.X(w1) @ qml.X(w2) @ qml.X(w3),
    qml.Y(w0) @ qml.X(w1) @ qml.Y(w2) @ qml.Y(w3),
    qml.Y(w0) @ qml.Y(w1) @ qml.X(w2) @ qml.Y(w3),
    qml.Y(w0) @ qml.Y(w1) @ qml.Y(w2) @ qml.X(w3),
]

op = qml.s_prod(scalar=coeffs, operator=obs)

In [ ]:
from pennylane.operation import gen_is_multi_term_hamiltonian

In [ ]:
doubleExcitation = qml.DoubleExcitation(0.1, wires=[0, 1, 2, 3])

In [ ]:
gen_is_multi_term_hamiltonian(doubleExcitation)

True

In [ ]:
type(doubleExcitation.generator())

pennylane.ops.qubit.hamiltonian.Hamiltonian